# Notebook to merge county level data with contact info

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
from tqdm import tqdm
import math

In [6]:
with open("../data/df_county_level_cached.pkl", 'rb') as f:
    county_df = pickle.load(f)

In [94]:
with open("countyFIPS_to_city.pkl", 'rb') as f:
    countyFIPS_to_city = pickle.load(f)

In [10]:
contact_info = pd.read_csv("all_state_numbers.csv", index_col = 0)

In [135]:
def find_contact(entry, contact_info):
    same_state = contact_info[contact_info["state"] == entry["StateNameAbbreviation"]]
    same_county = same_state[same_state["public_health"].apply(lambda x: entry["CountyName"] in x)]
    if same_county.shape[0] >= 1:
        return ", ".join(same_county["phone_number"]), ", ".join(same_county["public_health"])
    elif entry["countyFIPS"] in countyFIPS_to_city:
        city = countyFIPS_to_city[entry["countyFIPS"]]
        try:
            same_city = same_state[same_state["public_health"].apply(lambda x: city in x)]
            if same_city.shape[0] >= 1:
                return ", ".join(same_city["phone_number"]), ", ".join(same_city["public_health"])
            else:
                return None, None
        except:
            return None, None
    else:
        return None, None

In [136]:
find_contact(entry, contact_info)

('13343613743', 'Autauga County Health Department')

In [137]:
health_dept_names = []
contact_numbers = []
for idx, county in tqdm(county_df.iterrows()):
    result = find_contact(county, contact_info)
    health_dept_names.append(result[1])
    contact_numbers.append(result[0])

3114it [00:16, 183.81it/s]


In [138]:
county_df["HealthDeptName"] = health_dept_names
county_df["HealthDeptContact"] = contact_numbers

In [139]:
county_df[county_df["HealthDeptContact"].isna()].shape

(780, 7182)

In [144]:
county_df

,id,Header-FIPSStandCtyCode,EntityofFile,SecondaryEntityOfFile,DateofFile,DateofCreation,FileLength,StateName,StateNameAbbreviation,CountyName,...,#Hospitals,#ICU_beds,HeartDiseaseMortality,StrokeMortality,Smokers_Percentage,FracMale2017,#FTEHospitalTotal2017,mortality2015-17Estimated,HealthDeptName,HealthDeptContact
0,,01001,AHRF,01001,2019,19212.0,31661.0,Alabama,AL,Autauga,...,1,6,204.5,56.1,18.081557,0.486578,324.0,33.194755,Autauga County Health Department,13343613743
1,,01003,AHRF,01003,2019,19212.0,31661.0,Alabama,AL,Baldwin,...,3,51,183.2,41.9,17.489033,0.485472,2101.0,148.887537,Baldwin County Health Department,12519376935
2,,01005,AHRF,01005,2019,19212.0,31661.0,Alabama,AL,Barbour,...,1,5,220.4,49.0,21.999985,0.527701,181.0,18.483155,Barbour County Health Department,13346874808
3,,01007,AHRF,01007,2019,19212.0,31661.0,Alabama,AL,Bibb,...,1,0,225.5,57.2,19.114200,0.535469,145.0,15.381366,Bibb County Health Department,12059269702
4,,01009,AHRF,01009,2019,19212.0,31661.0,Alabama,AL,Blount,...,1,6,224.8,52.8,19.208672,0.493114,103.0,48.499442,Blount County Health Department,12052742120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,,56037,AHRF,56037,2019,19212.0,31661.0,Wyoming,WY,Sweetwater,...,2,10,193.4,32.5,18.073422,0.515459,605.0,16.269255,None,None
3110,,56039,AHRF,56039,2019,19212.0,31661.0,Wyoming,WY,Teton,...,1,6,90.7,28.9,14.546369,0.519020,604.0,3.253029,Teton County Public Health,13077336401
3111,,56041,AHRF,56041,2019,19212.0,31661.0,Wyoming,WY,Uinta,...,1,6,184.6,32.0,17.212675,0.504269,478.0,6.024624,Uinta County Public Health Nursing,13077898290
3112,,56043,AHRF,56043,2019,19212.0,31661.0,Wyoming,WY,Washakie,...,1,0,157.1,31.4,16.859400,0.505332,101.0,3.716395,Washakie County Public Health Nursing,13073473278


In [ ]:
county_df["StateNameAbbreviation"] == "NY"

In [141]:
county_df[county_df["HealthDeptContact"].isna()].sort_values(by = "PopulationEstimate2018", ascending = False)[["StateNameAbbreviation", "CountyName","PopulationEstimate2018", "countyFIPS"]][:20]

,StateNameAbbreviation,CountyName,PopulationEstimate2018,countyFIPS
1824,NY,Kings,2582830.0,36047
1841,NY,Queens,2278906.0,36081
1721,NV,Clark,2231647.0,32003
1803,NY,Bronx,1432132.0,36005
1954,NC,Wake,1092305.0,37183
293,DC,The District,702455.0,11001
616,IL,Lake,700832.0,17097
2418,TN,Davidson,692587.0,47037
220,CO,Arapahoe,651215.0,8005
2287,RI,Providence,636084.0,44007


In [143]:
countyFIPS_to_city[36081]

'Queens Village'

In [93]:
contact_info[contact_info["state"] == "TX"]

,state,public_health,phone_number
2939,TX,Abilene-Taylor County Public Health District,13256925600
2940,TX,Addison City Health Department,19724502821
2941,TX,Andrews County Health Department,14325241434
2942,TX,Angelina County/Cities Health District,19366330602
2943,TX,Austin Public Health,151297250101
...,...,...,...
3051,TX,Weatherford City Health Department,18175984149
3052,TX,Wichita Falls-Wichita County Public Health Dis...,19407617800
3053,TX,Williamson County and Cities Health District,15129433600
3054,TX,Wilson County Public Health and Safety,18303938503
